# Advanced Querying Mongo

Importing libraries and setting up connection

In [69]:
#pip install pymongo

In [100]:
from pymongo import MongoClient
import re

In [71]:
cursor = MongoClient()
cursor

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [72]:
str_con='localhost:27017'
cursor= MongoClient(str_con)
db=cursor.Ironhack

In [73]:
colect=db.CrunchBase
res=colect.find().limit(1)
list(res)[0].keys()

dict_keys(['_id', 'name', 'permalink', 'crunchbase_url', 'homepage_url', 'blog_url', 'blog_feed_url', 'twitter_username', 'category_code', 'number_of_employees', 'founded_year', 'founded_month', 'founded_day', 'deadpooled_year', 'tag_list', 'alias_list', 'email_address', 'phone_number', 'description', 'created_at', 'updated_at', 'overview', 'image', 'products', 'relationships', 'competitions', 'providerships', 'total_money_raised', 'funding_rounds', 'investments', 'acquisition', 'acquisitions', 'offices', 'milestones', 'video_embeds', 'screenshots', 'external_links', 'partners'])

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [102]:
# Your Code
query={'name':'Babelgum'}
select={ 'name': 1 , '_id': 0 , 'categoty_code' : 1}
list(colect.find(query,select))    

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [103]:
# Your Code
query={ 'number_of_employees' : {'$gt':5000}}
select={ 'name': 1 , '_id': 0}
list(colect.find(query,select).limit(20).sort('number_of_employees'))

[{'name': 'Nintendo'},
 {'name': 'Hexaware Technologies'},
 {'name': 'Facebook'},
 {'name': 'OpenText'},
 {'name': 'CPM Braxis'},
 {'name': 'LSI'},
 {'name': 'Microchip Technologies'},
 {'name': 'Mediaset'},
 {'name': 'Mindray Medical International'},
 {'name': 'Dentsu'},
 {'name': 'Atmel'},
 {'name': 'Tata Communications'},
 {'name': 'Baidu'},
 {'name': 'Acxiom'},
 {'name': 'Acxiom'},
 {'name': 'SRA International'},
 {'name': "Moody's"},
 {'name': 'ManTech'},
 {'name': 'Adobe Systems'},
 {'name': 'Broadcom'}]

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [198]:
query={'$and':[{'founded_year':{'$gte': 2000 }},{'founded_year' : { '$lte': 2005}}]}
select={'name':1 , '_id' :0, 'founded_year':1}
list(colect.find(query,select).sort('founded_year',-1).limit(3))

[{'name': 'Omnidrive', 'founded_year': 2005},
 {'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [99]:
# Your Code
def dinero(x):
    res=[]
    numero=1
    simbol=1
    for i in x:
        if i[0]=='$': simbol=1.3
        elif i[0]=='€': simbol=1.12
        if i[-1]=='K': numero=1e3
        elif i[-1]=='M': numero=1e6
        elif i[-1]=='B': numero=1e9
        n_col=re.findall('[0-9.]+', i)[0]
    res.append(int(float(n_col)*numero*simbol))
    return res
        

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [110]:
# Your Code
query={'$and':[{'number_of_employees':{'$lte':1000}},{'founded_year': {'$gte':2005}}]}
select={'name':1,'_id':0,'number_of_employees':1,'founded_year':1}
list(colect.find(query,select).sort('number_of_employees',-1).limit(10))

[{'name': 'Yodle', 'number_of_employees': 1000, 'founded_year': 2005},
 {'name': 'JinkoSolar Holding',
  'number_of_employees': 1000,
  'founded_year': 2006},
 {'name': 'Brain Bakery', 'number_of_employees': 1000, 'founded_year': 2006},
 {'name': 'Gumtree', 'number_of_employees': 1000, 'founded_year': 2005},
 {'name': 'Hello Travel', 'number_of_employees': 1000, 'founded_year': 2008},
 {'name': 'Yatra online', 'number_of_employees': 1000, 'founded_year': 2006},
 {'name': 'Box', 'number_of_employees': 950, 'founded_year': 2005},
 {'name': 'Relax Solutions Pvt Ltd',
  'number_of_employees': 900,
  'founded_year': 2006},
 {'name': 'Workday', 'number_of_employees': 900, 'founded_year': 2005},
 {'name': 'InMobi', 'number_of_employees': 900, 'founded_year': 2007}]

### 6. All the companies that don't include the `partners` field.

In [124]:
# Your Code
list(colect.find({'partners': {'$exists': False}}))

[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [197]:
# Your Code
query={'category_code':None}
select={'name':1.,'_id':0}
list(colect.find(query,select).limit(3))

[{'name': 'Collective'}, {'name': 'Snimmer'}, {'name': 'KoolIM'}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [196]:
# Your Code
query={'$and':[{'number_of_employees':{'$gte':100}},{'number_of_employees':{'$lte':1000}}]}
select={'name':1,'_id':0,'number_of_employees':1}
list(colect.find(query,select).limit(3))

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305}]

### 9. Order all the companies by their IPO price in a descending order.

In [83]:
# Your Code

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [195]:
# Your Code
list(colect.find({'number_of_employees':{'$gte':0}},{'name':1,'_id':0,'number_of_employees':1}).sort('number_of_employees',-1).limit(10))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [194]:
# Your Code
query={'$and':[{'founded_month': {'$gte':6}},{'founded_month':{'$lte':12}}]}
select_n_i={'name':1,'_id':0,'founded_month':1}
list(colect.find(query,select_n_i).sort('founded_month',-1).limit(3))

[{'name': 'Wesabe', 'founded_month': 12},
 {'name': 'Thoof', 'founded_month': 12},
 {'name': 'Kyte', 'founded_month': 12}]

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [193]:
# Your Code
query={'$and':[{'acquisition.price_amount':{"$gte":10000000}},{'founded_year':{"$gte":2000}}]}
select={"name":1,"_id":0,'acquisition.price_amount':1,'founded_year':1}
list(colect.find(query,select).sort('acquisition.price_amount',-1).limit(3))

[{'name': 'SuccessFactors',
  'founded_year': 2001,
  'acquisition': {'price_amount': 3400000000.0}},
 {'name': 'Sourcefire',
  'founded_year': 2001,
  'acquisition': {'price_amount': 2700000000.0}},
 {'name': 'Skype',
  'founded_year': 2003,
  'acquisition': {'price_amount': 2600000000.0}}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [192]:
# Your Code
query={'acquisition.acquired_year':{'$gte':2010}}
select={"name":1,"_id":0,'acquisition.acquiring_company':1}
list(colect.find(query,select).sort('acquisition.price_amount',-1).limit(3))

[{'name': 'T-Mobile',
  'acquisition': {'acquiring_company': {'name': 'AT&T', 'permalink': 'at-t'}}},
 {'name': 'Goodrich Corporation',
  'acquisition': {'acquiring_company': {'name': 'United Technologies',
    'permalink': 'united-technologies'}}},
 {'name': 'LSI',
  'acquisition': {'acquiring_company': {'name': 'Avago Technologies',
    'permalink': 'avago-technologies'}}}]

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [191]:
# Your Code
query={'founded_year':{'$ne':None}}
select={"name":1,"_id":0,'founded_year':1}
list(colect.find(query,select).sort('founded_year',-1).limit(3))

[{'name': 'Advaliant', 'founded_year': 2013},
 {'name': 'Wamba', 'founded_year': 2013},
 {'name': 'Fixya', 'founded_year': 2013}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [171]:
# Your Code
query={'founded_day':{'$gte':7}}
select={"name":1,"_id":0,'founded_day':1}
list(colect.find(query,select).sort('acquisition.price_amount',-1).limit(10))

[{'name': 'Siebel Systems', 'founded_day': 30},
 {'name': 'Yammer', 'founded_day': 10},
 {'name': 'Yammer', 'founded_day': 10},
 {'name': 'ngmoco', 'founded_day': 26},
 {'name': 'Audible', 'founded_day': 21},
 {'name': 'Adify', 'founded_day': 8},
 {'name': 'Zong', 'founded_day': 15},
 {'name': 'Topsy Labs', 'founded_day': 15},
 {'name': 'BreakingPoint Systems', 'founded_day': 28},
 {'name': 'Goodreads', 'founded_day': 15}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [90]:
# Your Code
query={'$and'}
select={"name":1,"_id":0,'acquisition.price_amount':1,'founded_year':1}
list(colect.find(query,select))

In [190]:
query={'$and':[{'category_code':'web'},{'number_of_employees':{'$gte':4000}}]}
select={"name":1,"_id":0,'category_code':1,'number_of_employees':1}
list(colect.find(query,select).sort('founded_year').limit(3))

[{'name': 'Los Angeles Times Media Group',
  'category_code': 'web',
  'number_of_employees': 10000},
 {'name': 'Experian', 'category_code': 'web', 'number_of_employees': 15500},
 {'name': 'AOL', 'category_code': 'web', 'number_of_employees': 8000}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [189]:
# Your Code
query={'$and':[{'acquisition.price_amount':{'$gte':1e7}},{'acquisition.price_currency_code':'EUR'}]}
select={"name":1,"_id":0,'acquisition.price_amount':1,'price_currency_code':1}
list(colect.find(query,select).sort('acquisition.price_amount',-1).limit(2))

[{'name': 'Apertio', 'acquisition': {'price_amount': 140000000}},
 {'name': 'Webedia', 'acquisition': {'price_amount': 70000000}}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [188]:
# Your Code
query={'$and':[{'acquisition.acquired_month':{'$gte':1}},{'acquisition.acquired_month':{'$lte':3}}]}
select={"name":1,"_id":0,'acquisition.acquired_month':1}
list(colect.find(query,select).limit(10))

[{'name': 'Kyte', 'acquisition': {'acquired_month': 1}},
 {'name': 'NetRatings', 'acquisition': {'acquired_month': 2}},
 {'name': 'blogTV', 'acquisition': {'acquired_month': 3}},
 {'name': 'Livestream', 'acquisition': {'acquired_month': 1}},
 {'name': 'iContact', 'acquisition': {'acquired_month': 2}},
 {'name': 'Coghead', 'acquisition': {'acquired_month': 2}},
 {'name': 'Dailymotion', 'acquisition': {'acquired_month': 2}},
 {'name': 'Netvibes', 'acquisition': {'acquired_month': 2}},
 {'name': 'Flickr', 'acquisition': {'acquired_month': 3}},
 {'name': 'BabyCenter', 'acquisition': {'acquired_month': 3}}]

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [186]:
# Your Code
query={'$and':[{'founded_year':{'$gte':2000}},{'founded_year':{'$lte':2010}},{'acquisition.acquired_year':{'$gte':2011}}]}
select={'name':1,'_id':0,'founded_year':1}
list(colect.find(query,select).sort('founded_year',-1).limit(2))

[{'name': 'Magento', 'founded_year': 2010},
 {'name': 'Trunkt', 'founded_year': 2009}]

### 20. All the companies that have been 'deadpooled' after the third year.

In [203]:
# Your Code
query={
    'founded_year': {'$exists': True},
    'deadpooled_year': {'$exists': True},
    '$expr': {'$eq': [{'$subtract': ['$deadpooled_year', '$founded_year']}, 3]}
}
select={'name':1,'_id':0,'founded_year':1,'deadpooled_year':1}
list(colect.find(query,select).limit(4))


[{'name': 'Omnidrive', 'founded_year': 2005, 'deadpooled_year': 2008},
 {'name': 'Jangl SMS', 'founded_year': 2005, 'deadpooled_year': 2008},
 {'name': 'TripHub', 'founded_year': 2005, 'deadpooled_year': 2008},
 {'name': 'EventVue', 'founded_year': 2007, 'deadpooled_year': 2010}]